## ERA V3 Session 13 Assignment - SmolLM2-135 model


1. Find out the details about SmolLM2-135 model. You won't get model.py free of cost online. You need to read its github, get yaml file for training, and download the 135 model, and then reverse engineer it. 
2. Training it for 5000 steps while predicting every 500 steps on what it utters. Now fully stop the model and save a checkpoint. Now load this checkpoint and train for 50 more steps.
3. Use all the speedups that we have used. 
4. Submit the GitHub link where I can see your README.md explaining the model definition. Parameter Calculation. Upload the model to Spaces as well and then share both the links.

### Defining the Model using LlamaConfig class

SmolLM2-135M model has ```is_llama_config = True``` as one of the parameters, which means it can be created using the LlamaConfig Class by inputing the same parameters as written in ```config_smollm2_135M.yaml```

In [2]:
# model.py
from transformers import LlamaConfig, LlamaForCausalLM, AutoTokenizer
import torch

def create_smollm2_model():
    """
    Constructs a SmoLLM2 model based on the provided configuration.

    Returns:
        tuple: A tuple containing the initialized model and tokenizer.
    """

    model_config = LlamaConfig(
        vocab_size=49152,
        hidden_size=576,
        intermediate_size=1536,
        num_hidden_layers=30,
        num_attention_heads=9,
        num_key_value_heads=3,
        hidden_act="silu",
        max_position_embeddings=2048,
        initializer_range=0.041666666666666664,
        rms_norm_eps=1.0e-05,
        # use_cache=True, As seen in training, this is not needed
        tie_word_embeddings=True,
        rope_theta=10000.0,
        rope_scaling=None,
        rope_interleaved=False,
        pretraining_tp=1,
        bos_token_id=0,
        eos_token_id=0,
        pad_token_id=None, #  pad_token_id is null in config, setting to None
    )

    model = LlamaForCausalLM(model_config)

    # Initialize weights with std from init_method if needed (Transformers usually handles initialization well)
    # init_std = 0.041666666666666664
    # You can add custom weight initialization here if required based on init_method.std

    # Set the dtype to bfloat16
    model.to(torch.bfloat16)

    # Load the tokenizer
    tokenizer_name_or_path = "HuggingFaceTB/cosmo2-tokenizer"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)


    return model, tokenizer

model, tokenizer = create_smollm2_model()
print("SmoLLM2 model and tokenizer successfully created!")
print(f"Model dtype: {next(model.parameters()).dtype}")
print(f"Tokenizer: {tokenizer.__class__.__name__} loaded from {tokenizer.name_or_path}")

c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SmoLLM2 model and tokenizer successfully created!
Model dtype: torch.bfloat16
Tokenizer: GPT2TokenizerFast loaded from HuggingFaceTB/cosmo2-tokenizer


### Model Architecture and parameter count

In [3]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm

In [4]:
tokenizer

GPT2TokenizerFast(name_or_path='HuggingFaceTB/cosmo2-tokenizer', vocab_size=49152, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|im_start|>', '<|im_end|>', '<repo_name>', '<reponame>', '<file_sep>', '<filename>', '<gh_stars>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, speci

In [5]:
model.config

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "silu",
  "hidden_size": 576,
  "initializer_range": 0.041666666666666664,
  "intermediate_size": 1536,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 9,
  "num_hidden_layers": 30,
  "num_key_value_heads": 3,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_interleaved": false,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "transformers_version": "4.44.1",
  "use_cache": true,
  "vocab_size": 49152
}

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params}')
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params}')

Total parameters: 134515008
Trainable parameters: 134515008


## Sample output generation

In [2]:
# Example usage (optional):
input_text = "Hello, world!"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated output: {decoded_output}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated output: Hello, world!immer Joseph params transactionDateTimeField plagiarism sniffShdis borders savage debit interfering polyunsaturatedothySh


## Loading input text and training the model

In [1]:
import os
with open("datasets/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [3]:
# train.py
import torch
from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelForCausalLM # Changed import for checkpointing
# from model import create_smollm2_model
import os

# --- 1. Hyperparameters and Configuration ---
TRAIN_STEPS = 5000
PREDICT_EVERY_STEPS = 500
CHECKPOINT_EVERY_STEPS = 500
SEQUENCE_LENGTH = 2048  # As defined in config - May need to reduce if OOM
MICRO_BATCH_SIZE = 8     # As defined in config - REDUCE THIS FIRST to fix OOM
LEARNING_RATE = 3e-4     # You can adjust, simplified from config for now
WARMUP_STEPS = 500      # Simplified warmup
CHECKPOINT_PATH = "smollm2-checkpoints"
INPUT_FILE = "datasets\input.txt"
PREDICTION_PROMPT = "To be or not to be," # A starting prompt for prediction

GRADIENT_ACCUMULATION_STEPS = 4 # --- Speedup 1: Gradient Accumulation --- Accumulate gradients over this many steps
USE_ACTIVATION_CHECKPOINTING = True # --- Speedup 2: Activation Checkpointing --- Enable or disable activation checkpointing

# --- ***MEMORY OPTIMIZATION - REDUCE THESE IF OOM ERROR PERSISTS*** ---
REDUCE_MICRO_BATCH_SIZE_FACTOR = 2 # --- Reduce Micro Batch Size --- Reduce micro_batch_size by this factor
# If you STILL get OOM, try reducing SEQUENCE_LENGTH_FACTOR (but micro_batch_size reduction is usually more effective first)
REDUCE_SEQUENCE_LENGTH_FACTOR = 1 # --- Reduce Sequence Length --- Reduce sequence length by this factor if needed

# --- 2. Adjusted Hyperparameters based on Reduction Factors ---
ADJUSTED_MICRO_BATCH_SIZE = MICRO_BATCH_SIZE // REDUCE_MICRO_BATCH_SIZE_FACTOR
if ADJUSTED_MICRO_BATCH_SIZE <= 0:
    ADJUSTED_MICRO_BATCH_SIZE = 1 # Ensure micro_batch_size is at least 1
ADJUSTED_SEQUENCE_LENGTH = SEQUENCE_LENGTH // REDUCE_SEQUENCE_LENGTH_FACTOR
if ADJUSTED_SEQUENCE_LENGTH <= 0:
    ADJUSTED_SEQUENCE_LENGTH = 64 # Ensure sequence_length is at least reasonably sized

MICRO_BATCH_SIZE = ADJUSTED_MICRO_BATCH_SIZE # Update micro batch size
SEQUENCE_LENGTH = ADJUSTED_SEQUENCE_LENGTH # Update sequence length


# --- 3. Create Checkpoint Directory ---
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# --- 4. Load Model and Tokenizer ---
model, tokenizer = create_smollm2_model()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# --- Speedup 3: Mixed Precision Training (bfloat16) --- Model is already in bfloat16 from model.py
# We are using bfloat16 dtype for model which speeds up training and reduces memory usage on compatible GPUs (like NVIDIA A100, H100)

if USE_ACTIVATION_CHECKPOINTING: # --- Enable activation checkpointing if flag is set ---
    model.gradient_checkpointing_enable() # Enable activation checkpointing from transformers

# --- 5. Prepare Dataset ---
print("Loading and tokenizing dataset...")
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    text = f.read()

tokenized_dataset = tokenizer(text, return_tensors="pt", truncation=False) # No truncation initially
input_ids = tokenized_dataset['input_ids']

# --- 6. Create Data Batches ---
def create_batches(input_ids, seq_len, micro_batch_size):
    num_tokens = input_ids.shape[1]
    num_batches = num_tokens // seq_len
    truncated_input_ids = input_ids[:, :num_batches * seq_len] # Truncate to fit full sequences based on potentially reduced SEQUENCE_LENGTH
    batched_input_ids = truncated_input_ids.reshape(-1, seq_len) # Reshape into sequences
    num_micro_batches = num_batches // micro_batch_size
    micro_batches = []
    for i in range(num_micro_batches):
        start_index = i * micro_batch_size
        end_index = (i + 1) * micro_batch_size
        batch = batched_input_ids[start_index:end_index]
        micro_batches.append(batch)
    return micro_batches

micro_batches = create_batches(input_ids.to(device), SEQUENCE_LENGTH, MICRO_BATCH_SIZE) # Using potentially reduced SEQUENCE_LENGTH and MICRO_BATCH_SIZE
print(f"Dataset prepared with {len(micro_batches)} micro-batches using micro_batch_size={MICRO_BATCH_SIZE} and sequence_length={SEQUENCE_LENGTH}.")


# --- 7. Optimizer and Scheduler ---
# --- Speedup 4: Fused AdamW Optimizer --- Using fused AdamW if available (often faster)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01, fused=True) # Using fused=True for potential speedup
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=TRAIN_STEPS // GRADIENT_ACCUMULATION_STEPS # Adjusted lr scheduler steps
) # Simplified scheduler

# --- imports ---
from tqdm import tqdm

# --- training loop ---
print("Starting training...")
model.train() # Set model to training mode
global_step = 0
accumulated_loss = 0 # To track loss over accumulation steps

# Initialize tqdm progress bar, total is global steps (weight updates)
progress_bar = tqdm(range(1, TRAIN_STEPS + 1), desc="Training", unit="step")

for step in progress_bar: # Wrap training loop with tqdm
    batch_index = (step - 1) % len(micro_batches) # Cycle through batches
    batch = micro_batches[batch_index]

    inputs = batch
    targets = torch.roll(batch, shifts=-1, dims=1) # Next token prediction

    outputs = model(inputs, labels=targets) # Labels for loss calculation
    loss = outputs.loss
    loss = loss / GRADIENT_ACCUMULATION_STEPS # --- Scale loss for gradient accumulation ---
    accumulated_loss += loss.item() # Accumulate loss for logging

    loss.backward()

    if step % GRADIENT_ACCUMULATION_STEPS == 0: # --- Update weights every GRADIENT_ACCUMULATION_STEPS ---
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1 # Increment global step only when weights are updated
        avg_loss = accumulated_loss / GRADIENT_ACCUMULATION_STEPS # Calculate average loss over accumulation steps
        accumulated_loss = 0 # Reset accumulated loss

        if global_step % 10 == 0: # Log loss every 10 global steps (after accumulation)
            # No need for separate print here, tqdm will handle logging

            # --- Update tqdm progress bar with current average loss ---
            progress_bar.set_postfix({"Avg Loss": f"{avg_loss:.4f}"})


    # --- 8. Prediction Interval ---
    if step % PREDICT_EVERY_STEPS == 0:
        model.eval() # Set model to evaluation mode
        print(f"\n--- Prediction at Micro-batch Step {step} (Global Step: {global_step}) ---") # Clarify step counts
        prompt_ids = tokenizer.encode(PREDICTION_PROMPT, return_tensors="pt").to(device)
        sample_outputs = model.generate(
            prompt_ids,
            max_length=len(prompt_ids[0]) + 50, # Generate up to 50 new tokens
            num_return_sequences=1,
            temperature=0.7, # Adjust temperature for creativity
            top_p=0.9,
        )
        predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
        print(f"Prompt: '{PREDICTION_PROMPT}'")
        print(f"Generated: '{predicted_text}'\n")
        model.train() # Set back to training mode

    # --- 9. Checkpoint Saving ---
    if step % CHECKPOINT_EVERY_STEPS == 0:
        checkpoint_file = os.path.join(CHECKPOINT_PATH, f"smollm2_checkpoint_step_{step}.pth") # Step here is still micro-batch step
        torch.save({
            'step': step, # Step here is still micro-batch step
            'global_step': global_step, # Saving global step as well (steps with weight updates)
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss.item(), # Save current micro-batch loss (last loss in accumulation)
        }, checkpoint_file)
        print(f"Checkpoint saved at Micro-batch Step {step} (Global Step {global_step}) to {checkpoint_file}") # Clarified step counts in checkpoint message

progress_bar.close() # Close progress bar when training finishes
print("Training finished!")

Loading and tokenizing dataset...
Dataset prepared with 41 micro-batches using micro_batch_size=4 and sequence_length=2048.
Starting training...


Training:  10%|▉         | 499/5000 [09:56<1:28:35,  1.18s/step, Avg Loss=1.9720]


--- Prediction at Micro-batch Step 500 (Global Step: 125) ---


c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a co

Prompt: 'To be or not to be,'
Generated: 'To be or not to be, dm emotionally Whole theware
:



::
:::::






























'



...


--- Prediction at Micro-batch Step 5000 (Global Step: 1250) ---


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt: 'To be or not to be,'
Generated: 'To be or not to be,,,,, I;, aw,
's thy now!
 it
 didy
 but! thee
 sweet,; did,,,, my to,
 did else and's; the if you
 will. all do'



Training: 100%|██████████| 5000/5000 [1:45:45<00:00,  1.27s/step, Avg Loss=0.1753]

Checkpoint saved at Micro-batch Step 5000 (Global Step 1250) to smollm2-checkpoints\smollm2_checkpoint_step_5000.pth
Training finished!


I have trained it for 5000 micro-steps (1250 weight-updating steps because of Gradient Accumulation)

## Loading Saved Checkpoint and re-training

To achieve the intended objective, we will load the checkpoint and further train it till 5000th weight-updating step (20000 total micro-steps)

In [3]:
RESUME_CHECKPOINT_PATH = "smollm2-checkpoints\smollm2_checkpoint_step_5000.pth"

# train.py
import torch
from transformers import get_linear_schedule_with_warmup # Changed import for checkpointing
# from model import create_smollm2_model
import os

# --- 1. Hyperparameters and Configuration ---
TARGET_GLOBAL_STEPS = 5000
PREDICT_EVERY_STEPS = 500
CHECKPOINT_EVERY_STEPS = 500
SEQUENCE_LENGTH = 2048  # As defined in config - May need to reduce if OOM
MICRO_BATCH_SIZE = 8     # As defined in config - REDUCE THIS FIRST to fix OOM
LEARNING_RATE = 3e-4     # You can adjust, simplified from config for now
WARMUP_STEPS = 500      # Simplified warmup
CHECKPOINT_PATH = "smollm2-checkpoints"
INPUT_FILE = "datasets\input.txt"
PREDICTION_PROMPT = "To be or not to be," # A starting prompt for prediction

GRADIENT_ACCUMULATION_STEPS = 4 # --- Speedup 1: Gradient Accumulation --- Accumulate gradients over this many steps
TARGET_TRAIN_STEPS = TARGET_GLOBAL_STEPS * GRADIENT_ACCUMULATION_STEPS # Adjusted total steps based on accumulation steps

USE_ACTIVATION_CHECKPOINTING = True # --- Speedup 2: Activation Checkpointing --- Enable or disable activation checkpointing

# --- ***MEMORY OPTIMIZATION - REDUCE THESE IF OOM ERROR PERSISTS*** ---
REDUCE_MICRO_BATCH_SIZE_FACTOR = 2 # --- Reduce Micro Batch Size --- Reduce micro_batch_size by this factor
# If you STILL get OOM, try reducing SEQUENCE_LENGTH_FACTOR (but micro_batch_size reduction is usually more effective first)
REDUCE_SEQUENCE_LENGTH_FACTOR = 1 # --- Reduce Sequence Length --- Reduce sequence length by this factor if needed

# --- 2. Adjusted Hyperparameters based on Reduction Factors ---
ADJUSTED_MICRO_BATCH_SIZE = MICRO_BATCH_SIZE // REDUCE_MICRO_BATCH_SIZE_FACTOR
if ADJUSTED_MICRO_BATCH_SIZE <= 0:
    ADJUSTED_MICRO_BATCH_SIZE = 1 # Ensure micro_batch_size is at least 1
ADJUSTED_SEQUENCE_LENGTH = SEQUENCE_LENGTH // REDUCE_SEQUENCE_LENGTH_FACTOR
if ADJUSTED_SEQUENCE_LENGTH <= 0:
    ADJUSTED_SEQUENCE_LENGTH = 64 # Ensure sequence_length is at least reasonably sized

MICRO_BATCH_SIZE = ADJUSTED_MICRO_BATCH_SIZE # Update micro batch size
SEQUENCE_LENGTH = ADJUSTED_SEQUENCE_LENGTH # Update sequence length


# --- 3. Create Checkpoint Directory ---
os.makedirs(CHECKPOINT_PATH, exist_ok=True)


# --- 4. Load Model and Tokenizer ---
model, tokenizer = create_smollm2_model()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# --- Speedup 3: Mixed Precision Training (bfloat16) --- Model is already in bfloat16 from model.py
# We are using bfloat16 dtype for model which speeds up training and reduces memory usage on compatible GPUs (like NVIDIA A100, H100)

if USE_ACTIVATION_CHECKPOINTING: # --- Enable activation checkpointing if flag is set ---
    model.gradient_checkpointing_enable() # Enable activation checkpointing from transformers

# --- 5. Prepare Dataset ---
print("Loading and tokenizing dataset...")
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    text = f.read()

tokenized_dataset = tokenizer(text, return_tensors="pt", truncation=False) # No truncation initially
input_ids = tokenized_dataset['input_ids']

# --- 8. Optimizer and Scheduler ---
# --- Speedup 4: Fused AdamW Optimizer --- Using fused AdamW if available (often faster)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01, fused=True) # Using fused=True for potential speedup
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=TARGET_TRAIN_STEPS // GRADIENT_ACCUMULATION_STEPS # Adjusted lr scheduler steps
) # Simplified scheduler

# --- 6. Load Checkpoint if `resume_checkpoint_path` is provided ---
initial_global_step = 0 # Track initial global step, default is 0 for new training
if RESUME_CHECKPOINT_PATH:
    print(f"Loading checkpoint from: {RESUME_CHECKPOINT_PATH}")
    checkpoint = torch.load(RESUME_CHECKPOINT_PATH, map_location=device, weights_only=False) # Load checkpoint to correct device
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    initial_global_step = checkpoint.get('global_step', 0) # Try to get global_step, default to 0 if not found
    print(f"Resuming training from global step: {initial_global_step}")
else:
    print("Starting training from scratch.")

start_step = initial_global_step * GRADIENT_ACCUMULATION_STEPS + 1 # Calculate micro-batch step to start from

# --- 7. Create Data Batches ---
def create_batches(input_ids, seq_len, micro_batch_size):
    num_tokens = input_ids.shape[1]
    num_batches = num_tokens // seq_len
    truncated_input_ids = input_ids[:, :num_batches * seq_len] # Truncate to fit full sequences based on potentially reduced SEQUENCE_LENGTH
    batched_input_ids = truncated_input_ids.reshape(-1, seq_len) # Reshape into sequences
    num_micro_batches = num_batches // micro_batch_size
    micro_batches = []
    for i in range(num_micro_batches):
        start_index = i * micro_batch_size
        end_index = (i + 1) * micro_batch_size
        batch = batched_input_ids[start_index:end_index]
        micro_batches.append(batch)
    return micro_batches

micro_batches = create_batches(input_ids.to(device), SEQUENCE_LENGTH, MICRO_BATCH_SIZE) # Using potentially reduced SEQUENCE_LENGTH and MICRO_BATCH_SIZE
print(f"Dataset prepared with {len(micro_batches)} micro-batches using micro_batch_size={MICRO_BATCH_SIZE} and sequence_length={SEQUENCE_LENGTH}.")


# --- imports ---
from tqdm import tqdm

# --- 9. Training Loop ---
print("Starting training...")
model.train() # Set model to training mode
global_step = initial_global_step # Initialize global_step from checkpoint or 0
accumulated_loss = 0 # To track loss over accumulation steps

# Initialize tqdm progress bar, total is remaining global steps
remaining_global_steps = TARGET_GLOBAL_STEPS - initial_global_step
if remaining_global_steps <= 0:
    print("Training already completed to target steps or beyond based on checkpoint.")
    exit()

progress_bar = tqdm(range(start_step, TARGET_TRAIN_STEPS + 1), desc="Training", unit="step", initial=start_step -1, total=TARGET_TRAIN_STEPS) # Initialize tqdm with start and total

for step in progress_bar: # Wrap training loop with tqdm
    batch_index = (step - 1) % len(micro_batches) # Cycle through batches
    batch = micro_batches[batch_index]

    inputs = batch
    targets = torch.roll(batch, shifts=-1, dims=1) # Next token prediction

    outputs = model(inputs, labels=targets) # Labels for loss calculation
    loss = outputs.loss
    loss = loss / GRADIENT_ACCUMULATION_STEPS # --- Scale loss for gradient accumulation ---
    accumulated_loss += loss.item() # Accumulate loss for logging

    loss.backward()

    if step % GRADIENT_ACCUMULATION_STEPS == 0: # --- Update weights every GRADIENT_ACCUMULATION_STEPS ---
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1 # Increment global step only when weights are updated
        avg_loss = accumulated_loss / GRADIENT_ACCUMULATION_STEPS # Calculate average loss over accumulation steps
        accumulated_loss = 0 # Reset accumulated loss

        if global_step % 10 == 0: # Log loss every 10 global steps (after accumulation)
            # No need for separate print here, tqdm will handle logging

            # --- Update tqdm progress bar with current average loss ---
            progress_bar.set_postfix({"Avg Loss": f"{avg_loss:.4f}"})


    # --- 8. Prediction Interval ---
    if step % PREDICT_EVERY_STEPS == 0:
        model.eval() # Set model to evaluation mode
        print(f"\n--- Prediction at Micro-batch Step {step} (Global Step: {global_step}) ---") # Clarify step counts
        prompt_ids = tokenizer.encode(PREDICTION_PROMPT, return_tensors="pt").to(device)
        sample_outputs = model.generate(
            prompt_ids,
            max_length=len(prompt_ids[0]) + 50, # Generate up to 50 new tokens
            num_return_sequences=1,
            temperature=0.7, # Adjust temperature for creativity
            top_p=0.9,
        )
        predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
        print(f"Prompt: '{PREDICTION_PROMPT}'")
        print(f"Generated: '{predicted_text}'\n")
        model.train() # Set back to training mode

    # --- 9. Checkpoint Saving ---
    if step % CHECKPOINT_EVERY_STEPS == 0:
        checkpoint_file = os.path.join(CHECKPOINT_PATH, f"smollm2_checkpoint_step_{step}.pth") # Step here is still micro-batch step
        torch.save({
            'step': step, # Step here is still micro-batch step
            'global_step': global_step, # Saving global step as well (steps with weight updates)
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss.item(), # Save current micro-batch loss (last loss in accumulation)
        }, checkpoint_file)
        print(f"Checkpoint saved at Micro-batch Step {step} (Global Step {global_step}) to {checkpoint_file}") # Clarified step counts in checkpoint message

progress_bar.close() # Close progress bar when training finishes
print("Training finished!")

Loading and tokenizing dataset...
Loading checkpoint from: smollm2-checkpoints\smollm2_checkpoint_step_5000.pth
Resuming training from global step: 1250
Dataset prepared with 41 micro-batches using micro_batch_size=4 and sequence_length=2048.
Starting training...


Training:  27%|██▋       | 5499/20000 [08:44<4:14:06,  1.05s/step, Avg Loss=0.0933]c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not se


--- Prediction at Micro-batch Step 5500 (Global Step: 1375) ---
Prompt: 'To be or not to be,'
Generated: 'To be or not to be,,,,, I:
 most low o, a wife
 mine thenoth up to lives
 your power your, itt
y of.
BDEL from
-?
N: who it not pray,; not'
'



Training:  28%|██▊       | 5500/20000 [08:48<7:41:58,  1.91s/step, Avg Loss=0.0933]

Checkpoint saved at Micro-batch Step 5500 (Global Step 1375) to smollm2-checkpoints\smollm2_checkpoint_step_5500.pth


...

Checkpoint saved at Micro-batch Step 19500 (Global Step 4875) to smollm2-checkpoints\smollm2_checkpoint_step_19500.pth


Training: 100%|█████████▉| 19999/20000 [4:33:47<00:01,  1.09s/step, Avg Loss=0.0168]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



--- Prediction at Micro-batch Step 20000 (Global Step: 5000) ---
Prompt: 'To be or not to be,'
Generated: 'To be or not to be,,,,, Iar,'t
 dis did your you
 are, will not
yTh-, am but
y be name with what will no.' my:
 with, will,, a and's,
--itor'



Training: 100%|██████████| 20000/20000 [4:33:49<00:00,  1.10s/step, Avg Loss=0.0168]

Checkpoint saved at Micro-batch Step 20000 (Global Step 5000) to smollm2-checkpoints\smollm2_checkpoint_step_20000.pth
Training finished!


As we can observe, model loaded from saved checkpoint showed average error loss start from the point where we stopped training before, showing that the **model checkpoint were saved correctly**

## Model Prediction 

SmolLLM-125M model after 5000 Steps (20000 Micro Steps)

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from model import create_smollm2_model # Assuming model.py is in the same directory

def generate_prediction(checkpoint_path, input_text, max_new_tokens=50, temperature=0.7, top_p=0.9):
    """
    Loads a model from a checkpoint and generates text based on an input prompt.

    Args:
        checkpoint_path (str): Path to the saved model checkpoint file (e.g., 'smollm2-checkpoints/smollm2_checkpoint_step_5000.pth').
        input_text (str): The text prompt to start generation from (e.g., "To be or not to be,").
        max_new_tokens (int): Maximum number of new tokens to generate.
        temperature (float): Sampling temperature for generation (higher values more creative, lower more deterministic).
        top_p (float): Top-p sampling parameter.

    Returns:
        str: The generated text.
    """
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # 1. Load Model and Tokenizer (same as in training)
    model, tokenizer = create_smollm2_model() # Use the same model creation function
    model.to(device)

    # 2. Load Model State from Checkpoint
    print(f"Loading checkpoint from: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device) # Load checkpoint to correct device
    model.load_state_dict(checkpoint['model_state_dict']) # Only load model weights

    # 3. Set Model to Evaluation Mode
    model.eval()

    # 4. Tokenize Input Text
    prompt_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # 5. Generate Prediction
    print(f"Generating text with prompt: '{input_text}'")
    sample_outputs = model.generate(
        prompt_ids,
        max_length=len(prompt_ids[0]) + max_new_tokens,
        num_return_sequences=1,
        temperature=temperature,
        top_p=top_p,
    )

    # 6. Decode and Return Generated Text
    predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    return predicted_text

if __name__ == "__main__":
    checkpoint_path = "smollm2-checkpoints\smollm2_checkpoint_step_20000.pth" # --- REPLACE WITH YOUR ACTUAL CHECKPOINT PATH ---
    input_prompt = "O art thou " # --- REPLACE WITH YOUR DESIRED PROMPT ---

    generated_text = generate_prediction(checkpoint_path, input_prompt)

    print(f"\nPrompt: '{input_prompt}'")
    print(f"Generated: '{generated_text}'")


Loading checkpoint from: smollm2-checkpoints\smollm2_checkpoint_step_20000.pth


C:\Users\Saish Shetty\AppData\Local\Temp\ipykernel_36572\1492698466.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_locatio

Generating text with prompt: 'O art thou '

Prompt: 'O art thou '
Generated: 'O art thou  ams?', leave son
 do th will the from thee one mine, there hear?
 will theyest thy but out sh as unto no
 bro one hisal'd' so; to you at
 brother not thees thesehouse'


## Current observations

1. We have trained the model for 5000 Global Steps, but average loss value has stayed consistent at around 0.0168
2. Now we will further train the model for further with smaller learning rate (old LR: 3e-4, new LR: 3e-5)


In [2]:
RESUME_CHECKPOINT_PATH = "smollm2-checkpoints\smollm2_checkpoint_step_20000.pth"

# train.py
import torch
from transformers import get_linear_schedule_with_warmup # Changed import for checkpointing
# from model import create_smollm2_model
import os

# --- 1. Hyperparameters and Configuration ---
TARGET_GLOBAL_STEPS = 5050
PREDICT_EVERY_STEPS = 100
CHECKPOINT_EVERY_STEPS = 200
SEQUENCE_LENGTH = 2048  # As defined in config - May need to reduce if OOM
MICRO_BATCH_SIZE = 8     # As defined in config - REDUCE THIS FIRST to fix OOM
LEARNING_RATE = 3e-5     # You can adjust, simplified from config for now
WARMUP_STEPS = 250      # Simplified warmup
CHECKPOINT_PATH = "smollm2-checkpoints"
INPUT_FILE = "datasets\input.txt"
PREDICTION_PROMPT = "To be or not to be," # A starting prompt for prediction

GRADIENT_ACCUMULATION_STEPS = 4 # --- Speedup 1: Gradient Accumulation --- Accumulate gradients over this many steps
TARGET_TRAIN_STEPS = TARGET_GLOBAL_STEPS * GRADIENT_ACCUMULATION_STEPS # Adjusted total steps based on accumulation steps

USE_ACTIVATION_CHECKPOINTING = True # --- Speedup 2: Activation Checkpointing --- Enable or disable activation checkpointing

# --- ***MEMORY OPTIMIZATION - REDUCE THESE IF OOM ERROR PERSISTS*** ---
REDUCE_MICRO_BATCH_SIZE_FACTOR = 2 # --- Reduce Micro Batch Size --- Reduce micro_batch_size by this factor
# If you STILL get OOM, try reducing SEQUENCE_LENGTH_FACTOR (but micro_batch_size reduction is usually more effective first)
REDUCE_SEQUENCE_LENGTH_FACTOR = 1 # --- Reduce Sequence Length --- Reduce sequence length by this factor if needed

# --- 2. Adjusted Hyperparameters based on Reduction Factors ---
ADJUSTED_MICRO_BATCH_SIZE = MICRO_BATCH_SIZE // REDUCE_MICRO_BATCH_SIZE_FACTOR
if ADJUSTED_MICRO_BATCH_SIZE <= 0:
    ADJUSTED_MICRO_BATCH_SIZE = 1 # Ensure micro_batch_size is at least 1
ADJUSTED_SEQUENCE_LENGTH = SEQUENCE_LENGTH // REDUCE_SEQUENCE_LENGTH_FACTOR
if ADJUSTED_SEQUENCE_LENGTH <= 0:
    ADJUSTED_SEQUENCE_LENGTH = 64 # Ensure sequence_length is at least reasonably sized

MICRO_BATCH_SIZE = ADJUSTED_MICRO_BATCH_SIZE # Update micro batch size
SEQUENCE_LENGTH = ADJUSTED_SEQUENCE_LENGTH # Update sequence length


# --- 3. Create Checkpoint Directory ---
os.makedirs(CHECKPOINT_PATH, exist_ok=True)


# --- 4. Load Model and Tokenizer ---
model, tokenizer = create_smollm2_model()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# --- Speedup 3: Mixed Precision Training (bfloat16) --- Model is already in bfloat16 from model.py
# We are using bfloat16 dtype for model which speeds up training and reduces memory usage on compatible GPUs (like NVIDIA A100, H100)

if USE_ACTIVATION_CHECKPOINTING: # --- Enable activation checkpointing if flag is set ---
    model.gradient_checkpointing_enable() # Enable activation checkpointing from transformers

# --- 5. Prepare Dataset ---
print("Loading and tokenizing dataset...")
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    text = f.read()

tokenized_dataset = tokenizer(text, return_tensors="pt", truncation=False) # No truncation initially
input_ids = tokenized_dataset['input_ids']

# --- 8. Optimizer and Scheduler ---
# --- Speedup 4: Fused AdamW Optimizer --- Using fused AdamW if available (often faster)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01, fused=True) # Using fused=True for potential speedup
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=TARGET_TRAIN_STEPS // GRADIENT_ACCUMULATION_STEPS # Adjusted lr scheduler steps
) # Simplified scheduler

# --- 6. Load Checkpoint if `resume_checkpoint_path` is provided ---
initial_global_step = 0 # Track initial global step, default is 0 for new training
if RESUME_CHECKPOINT_PATH:
    print(f"Loading checkpoint from: {RESUME_CHECKPOINT_PATH}")
    checkpoint = torch.load(RESUME_CHECKPOINT_PATH, map_location=device, weights_only=False) # Load checkpoint to correct device
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    initial_global_step = checkpoint.get('global_step', 0) # Try to get global_step, default to 0 if not found
    print(f"Resuming training from global step: {initial_global_step}")
else:
    print("Starting training from scratch.")

start_step = initial_global_step * GRADIENT_ACCUMULATION_STEPS + 1 # Calculate micro-batch step to start from

# --- 7. Create Data Batches ---
def create_batches(input_ids, seq_len, micro_batch_size):
    num_tokens = input_ids.shape[1]
    num_batches = num_tokens // seq_len
    truncated_input_ids = input_ids[:, :num_batches * seq_len] # Truncate to fit full sequences based on potentially reduced SEQUENCE_LENGTH
    batched_input_ids = truncated_input_ids.reshape(-1, seq_len) # Reshape into sequences
    num_micro_batches = num_batches // micro_batch_size
    micro_batches = []
    for i in range(num_micro_batches):
        start_index = i * micro_batch_size
        end_index = (i + 1) * micro_batch_size
        batch = batched_input_ids[start_index:end_index]
        micro_batches.append(batch)
    return micro_batches

micro_batches = create_batches(input_ids.to(device), SEQUENCE_LENGTH, MICRO_BATCH_SIZE) # Using potentially reduced SEQUENCE_LENGTH and MICRO_BATCH_SIZE
print(f"Dataset prepared with {len(micro_batches)} micro-batches using micro_batch_size={MICRO_BATCH_SIZE} and sequence_length={SEQUENCE_LENGTH}.")


# --- imports ---
from tqdm import tqdm

# --- 9. Training Loop ---
print("Starting training...")
model.train() # Set model to training mode
global_step = initial_global_step # Initialize global_step from checkpoint or 0
accumulated_loss = 0 # To track loss over accumulation steps

# Initialize tqdm progress bar, total is remaining global steps
remaining_global_steps = TARGET_GLOBAL_STEPS - initial_global_step
if remaining_global_steps <= 0:
    print("Training already completed to target steps or beyond based on checkpoint.")
    exit()

progress_bar = tqdm(range(start_step, TARGET_TRAIN_STEPS + 1), desc="Training", unit="step", initial=start_step -1, total=TARGET_TRAIN_STEPS) # Initialize tqdm with start and total

for step in progress_bar: # Wrap training loop with tqdm
    batch_index = (step - 1) % len(micro_batches) # Cycle through batches
    batch = micro_batches[batch_index]

    inputs = batch
    targets = torch.roll(batch, shifts=-1, dims=1) # Next token prediction

    outputs = model(inputs, labels=targets) # Labels for loss calculation
    loss = outputs.loss
    loss = loss / GRADIENT_ACCUMULATION_STEPS # --- Scale loss for gradient accumulation ---
    accumulated_loss += loss.item() # Accumulate loss for logging

    loss.backward()

    if step % GRADIENT_ACCUMULATION_STEPS == 0: # --- Update weights every GRADIENT_ACCUMULATION_STEPS ---
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1 # Increment global step only when weights are updated
        avg_loss = accumulated_loss / GRADIENT_ACCUMULATION_STEPS # Calculate average loss over accumulation steps
        accumulated_loss = 0 # Reset accumulated loss

        if global_step % 10 == 0: # Log loss every 10 global steps (after accumulation)
            # No need for separate print here, tqdm will handle logging

            # --- Update tqdm progress bar with current average loss ---
            progress_bar.set_postfix({"Avg Loss": f"{avg_loss:.4f}"})


    # --- 8. Prediction Interval ---
    if step % PREDICT_EVERY_STEPS == 0:
        model.eval() # Set model to evaluation mode
        print(f"\n--- Prediction at Micro-batch Step {step} (Global Step: {global_step}) ---") # Clarify step counts
        prompt_ids = tokenizer.encode(PREDICTION_PROMPT, return_tensors="pt").to(device)
        sample_outputs = model.generate(
            prompt_ids,
            max_length=len(prompt_ids[0]) + 50, # Generate up to 50 new tokens
            num_return_sequences=1,
            temperature=0.7, # Adjust temperature for creativity
            top_p=0.9,
        )
        predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
        print(f"Prompt: '{PREDICTION_PROMPT}'")
        print(f"Generated: '{predicted_text}'\n")
        model.train() # Set back to training mode

    # --- 9. Checkpoint Saving ---
    if step % CHECKPOINT_EVERY_STEPS == 0:
        checkpoint_file = os.path.join(CHECKPOINT_PATH, f"smollm2_checkpoint_step_{step}.pth") # Step here is still micro-batch step
        torch.save({
            'step': step, # Step here is still micro-batch step
            'global_step': global_step, # Saving global step as well (steps with weight updates)
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss.item(), # Save current micro-batch loss (last loss in accumulation)
        }, checkpoint_file)
        print(f"Checkpoint saved at Micro-batch Step {step} (Global Step {global_step}) to {checkpoint_file}") # Clarified step counts in checkpoint message

progress_bar.close() # Close progress bar when training finishes
print("Training finished!")

Loading and tokenizing dataset...
Loading checkpoint from: smollm2-checkpoints\smollm2_checkpoint_step_20000.pth
Resuming training from global step: 5000
Dataset prepared with 41 micro-batches using micro_batch_size=4 and sequence_length=2048.
Starting training...


Training: 100%|█████████▉| 20099/20200 [02:12<02:15,  1.34s/step, Avg Loss=0.0175]


--- Prediction at Micro-batch Step 20100 (Global Step: 5025) ---


c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Saish Shetty\.conda\envs\eraenv\lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a co

Prompt: 'To be or not to be,'
Generated: 'To be or not to be,,,,, Iar,'t
 dis did your you
 are, will not
yTh-, am buted,y more my
 treason- to make
 it at andy
 time no, the will at, I so'



Training: 100%|█████████▉| 20199/20200 [04:30<00:01,  1.34s/step, Avg Loss=0.0171]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



--- Prediction at Micro-batch Step 20200 (Global Step: 5050) ---
Prompt: 'To be or not to be,'
Generated: 'To be or not to be,,,,, Iar,'t
 dis did your you
 are, will not
yTh-, am buted,y more my
 treason- to make
 it at andy
 time no, the will at, I so'



Training: 100%|██████████| 20200/20200 [04:32<00:00,  1.36s/step, Avg Loss=0.0171]

Checkpoint saved at Micro-batch Step 20200 (Global Step 5050) to smollm2-checkpoints\smollm2_checkpoint_step_20200.pth
Training finished!


Final Average loss achieved: **0.0171** 

- Reducing LR rate by 10 did not significantly reduced model loss value

### Prediction using the updated model

In [9]:
checkpoint_path = "smollm2-checkpoints\smollm2_checkpoint_step_20200.pth" # --- REPLACE WITH YOUR ACTUAL CHECKPOINT PATH ---
input_prompt = "O art thou " # --- REPLACE WITH YOUR DESIRED PROMPT ---

generated_text = generate_prediction(checkpoint_path, input_prompt)

print(f"\nPrompt: '{input_prompt}'")
print(f"Generated: '{generated_text}'")

Loading checkpoint from: smollm2-checkpoints\smollm2_checkpoint_step_20200.pth


C:\Users\Saish Shetty\AppData\Local\Temp\ipykernel_36572\1492698466.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_locatio

Generating text with prompt: 'O art thou '

Prompt: 'O art thou '
Generated: 'O art thou  ams?', leave son
 do th will the from thee one mine, there hear?
 will theyest thy but out sh as unto no
 bro one hisal'd' so; to you at
 brother not thees thesehouse'
